# Первая модель 9 декабря 2025

## 1. Импорт библиотек и загрузка данных
Раздел «Imports & Data Loading»:  
Импорты  
Загрузка train 
Быстрый .head()  
Проверка формы датасета  
Краткие описания колонок (комментариями)

### Импорты

In [2]:
import os

import pandas as  pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler


### Загрузка train и Быстрый .head()

In [3]:
data_path = '../../Data/Kaggle/titanic/'

files = os.listdir(data_path)
for i in range(len(files)):
    print(f'file {i} - {files[i]}')

file 0 - test.csv
file 1 - train.csv
file 2 - gender_submission.csv


In [ ]:
df_train_row = pd.read_csv(f'{data_path}{files[1]}')
df_train_row.head()

### Краткие описания колонок (комментариями)

PassengerId — уникальный идентификатор пассажира, просто индекс.  
Survived — целевая переменная: 0 = погиб, 1 = выжил.  
Pclass — класс обслуживания: 1-й (дорогой), 2-й, 3-й (дешёвый).  
Name — имя (можно извлечь звание/титул: Mr, Miss, Mrs, etc.).  
Sex — пол (ключевой фактор выживания).  
Age — возраст (много пропусков → важна обработка).  
SibSp — количество родственников по боковой линии (братья, сёстры, супруги).  
Parch — количество родителей и детей.  
Ticket — номер билета (можно делать группировки).  
Fare — стоимость билета (важно для богатых → выше шанс выжить).  
Cabin — каюта (почти всё пропущено, но буква палубы полезна).  
Embarked — порт отправления: S, C, Q.  
  
  
Так какбудет использоваться логистическая регресссия, то порядковая переменная будет обрабатываться как номинальная
До заполнения пропусков нужно обработать выбросы

## 2. Анализ данных и обработка признаков

Общие статитики по датасету  
Анализ численных признаков  
Обработка числовых признаков  
Анализ нечисловых признаков  
Обработка нечисловых признаков  
Создание новых признкаов

### Общие статитики по датасету

In [ ]:
#Проверка на пропуски

df_train_row.info()

In [ ]:
df_train_row.isna().sum()/df_train_row.shape[0]*100

Cabin нужно удалить так как пропусков больше

In [ ]:
# Статистики

display(df_train_row.describe())
display(df_train_row.describe(include=['object']))

In [ ]:
#Создание DataFrame для препроцессинга
df_processed = df_train_row.copy()

### Распределение переменных по типу 

In [ ]:
feature_id = 'PassengerId'
target_name = 'Survived'
list_num_ftrs = [	
     'Age'
    ,'SibSp'
    ,'Parch'
    ,'Fare'
]

list_num_with_outliers = [
     'Age'
    ,'Fare'
]

list_not_num_ftrs = [
     'Pclass'
    ,'Name'
    ,'Sex'
    ,'Ticket'
    ,'Cabin'
    ,'Embarked'

]

list_not_num_ftrs_empty = [
     'Cabin'
]
rare_num_col = [
     'SibSp'
    ,'Parch'
]

if set(df_train_row.columns) == set(list_num_ftrs + list_not_num_ftrs + [feature_id, target_name]):
    print('Все переменные распределены')
else:
    print('Переменные не распределены')

In [ ]:
display(df_train_row[list_num_ftrs].info())
display(df_train_row[list_not_num_ftrs].info())

### 2.1. Анализ численных признаков

Идея заполняем выбросы нижней и верхней границей межквартильного размаха

In [ ]:
#Аналз распределений

for col in list_num_ftrs:
    plt.figure(figsize=(6, 4), dpi = 100)
    sns.boxenplot(data=df_train_row, x=target_name, y=col)
    plt.title(f"Распределение {col} по {target_name}")
    plt.show()

### 2.2 Заполняем пропуски

In [ ]:
#Пайплайны для разынх типов

numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
])

categorical_pipe_exceptions = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value='empty')),
])  
imputer = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, list_num_ftrs),
        ("cat", categorical_pipe, list(
            set(list_not_num_ftrs)-set(list_not_num_ftrs_empty)
        )),
        ("cat_exp", categorical_pipe_exceptions, list_not_num_ftrs_empty),
    ],
    remainder="passthrough"
)

imputer.set_output(transform="pandas")

In [ ]:
display(df_train_row.isna().sum()/df_train_row.shape[0]*100)
       
imputer.fit(df_train_row)
df_processed = pd.DataFrame(imputer.transform(df_train_row) )

df_processed.columns = [
    col.split('__', 1)[1] if "__" in col else col
    for col in df_processed.columns
]

display(df_processed.isna().sum()/df_train_row.shape[0]*100)

df_processed[list_not_num_ftrs_empty[0]].value_counts()

In [ ]:
#Создам отдельный класс, для обработки выбросов в числовых переменных с использованием межквартильного размаха

class ClipOutliersIQR(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None, k=1.5):
        self.columns = columns
        self.k = k
        self.bounds_ = {} #Здесь будут сохранятся границы для конкретного столбца

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        for col in self.columns:
            q1 = X[col].quantile(0.25)
            q3 = X[col].quantile(0.75)
            IQR = q3 - q1
            upper = q3 + self.k * IQR
            lower = q1 - self.k * IQR
            self.bounds_[col] = (lower, upper)

        return self

    
    def transform(self, X):
        X = pd.DataFrame(X).copy()
        for col, (lower, upper) in self.bounds_.items():
            X[col] = X[col].clip(lower, upper)

        return X

In [ ]:
clip_outlier_num = ClipOutliersIQR(columns=list_num_with_outliers)

df_processed = clip_outlier_num.fit_transform(df_processed)

df_processed

In [ ]:
# Распределеиня после выбросов:
#Аналз распределений

for col in list_num_ftrs:
    plt.figure(figsize=(6, 4), dpi = 100)
    sns.boxenplot(data=df_processed, x=target_name, y=col)
    plt.title(f"Распределение {col} по {target_name}")
    plt.show()

In [ ]:
df_processed['Parch'].value_counts(normalize=True)*100

In [ ]:
print(type(df_train_row['SibSp'].value_counts(normalize=True)))
rare_index = df_train_row['SibSp'].value_counts(normalize=True)*100<1
set(df_train_row['SibSp'].value_counts()[rare_index].index)

Для Parch и SibSp создадим категорию другие, куда попадут классы с менее 1 процентом.

In [ ]:
class RareCategoriesGrouper(BaseEstimator, TransformerMixin):
    def __init__(self, columns, min_freq=0.01, label_rare_categoty="other"):
        self.columns = columns
        self.min_freq = min_freq
        self.label_rare_categoty = label_rare_categoty
        self.rare_categories_ = {} #{column :set(rare_categories)}

        
    def fit(self, X, y=None):
        self.X = pd.DataFrame(X).copy()
        for col in self.columns:
            rare_index = X[col].value_counts(normalize=True)*100<1
            set(X[col].value_counts()[rare_index].index)
            self.rare_categories_[col] = set(X[col].value_counts()[rare_index].index)
            
        return self

    
    def transform(self, X):
        self.X = pd.DataFrame(X).copy()
        for col, rare_values in self.rare_categories_.items():
            X[col] = X[col].where(~X[col].isin(rare_values), self.label_rare_categoty)
            X[col] = X[col].apply(str)
        return X

In [ ]:
rarecategoriesgrouper = RareCategoriesGrouper(columns=rare_num_col)

df_processed = rarecategoriesgrouper.fit_transform(df_processed)

df_processed

In [ ]:
df_processed['Parch'].value_counts(normalize=True)*100

### Анализ нечисловых признаков

In [67]:
df_processed[list_not_num_ftrs]

,Pclass,Name,Sex,Ticket,Cabin,Embarked
0,3,mr,male,other,empty,S
1,1,mrs,female,other,C85,C
2,3,miss,female,other,empty,S
3,1,mrs,female,other,C123,S
4,3,mr,male,other,empty,S
...,...,...,...,...,...,...
886,2,other,male,other,empty,S
887,1,miss,female,other,B42,S
888,3,miss,female,other,empty,S
889,1,mr,male,other,C148,C


#### Анализ Pclass

Ничего делать не нужно просто OHE

In [ ]:
df_processed['Pclass'].value_counts()

#### Анализ Name

Выделяем признаки замужества и пола

In [ ]:
df_processed['Name'].apply(lambda x: [part for part in
                                      str(x).lower()
                                          .replace('.', '')
                                          .replace(',', '')
                                          .strip()
                                          .split()]).explode().value_counts()

In [ ]:
def prefix_name(name):
    list_name_prfx = [part for part in str(name).lower().replace('.', '').replace(',', '').strip().split()]
    if 'mr' in list_name_prfx:
        return 'mr'
    elif 'miss' in list_name_prfx:
        return 'miss'
    elif 'mrs' in list_name_prfx:
        return 'mrs'
    else:
        return 'other'

In [33]:
df_processed['Name'] = df_processed['Name'].apply(prefix_name)
df_processed['Name'].value_counts(normalize=True)*100

Name
mr       58.024691
miss     20.426487
mrs      14.029181
other     7.519641
Name: proportion, dtype: float64

#### Анализ Sex

Тут чисто OHE

In [69]:
df_processed['Sex'].value_counts(normalize=True)*100

Sex
male      64.758698
female    35.241302
Name: proportion, dtype: float64

#### Анализ Ticket

Создадим два поля длина билета и префикс билета для OHE

In [65]:
df_processed['Ticket_len'] = df_processed['Ticket'].apply(lambda x: len(str(x)))

In [58]:
import re
def remove_nbrs(string):
    return re.sub(r'\d+', '__', string)

df_processed['Ticket'].apply(lambda x: remove_nbrs(str(x)))

0           A/__ __
1             PC __
2      STON/O__. __
3                __
4                __
           ...     
886              __
887              __
888        W./C. __
889              __
890              __
Name: Ticket, Length: 891, dtype: object

In [64]:
df_processed['Ticket'].apply(lambda x: remove_nbrs(str(x)).lower().replace('.','').split()[0].replace('/','')[:2]).value_counts(normalize=True)*100

Ticket
__    74.186308
pc     6.734007
ca     4.713805
a_     3.142536
so     3.030303
st     2.020202
sc     1.907969
wc     1.122334
fc     0.673401
c      0.561167
pp     0.561167
li     0.448934
we     0.336700
sw     0.224467
sp     0.112233
fa     0.112233
as     0.112233
Name: proportion, dtype: float64

In [66]:
rarecategoriesgrouper_cat = RareCategoriesGrouper(columns=['Ticket'])

df_processed = rarecategoriesgrouper_cat.fit_transform(df_processed)

df_processed

,Age,SibSp,Parch,Fare,Embarked,Ticket,Name,Pclass,Sex,Cabin,PassengerId,Survived,Ticket_len
0,22.0,1.0,0.0,7.2500,S,other,mr,3,male,empty,1,0,9
1,38.0,1.0,0.0,65.6344,C,other,mrs,1,female,C85,2,1,8
2,26.0,0.0,0.0,7.9250,S,other,miss,3,female,empty,3,1,16
3,35.0,1.0,0.0,53.1000,S,other,mrs,1,female,C123,4,1,6
4,35.0,0.0,0.0,8.0500,S,other,mr,3,male,empty,5,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,27.0,0.0,0.0,13.0000,S,other,other,2,male,empty,887,0,6
887,19.0,0.0,0.0,30.0000,S,other,miss,1,female,B42,888,1,6
888,28.0,1.0,2.0,23.4500,S,other,miss,3,female,empty,889,0,10
889,26.0,0.0,0.0,30.0000,C,other,mr,1,male,C148,890,1,6


#### Анализ Cabin

In [74]:
df_processed['Cabin'].apply(lambda x: 'empty' if str(x)=='empty' else str(x)[0]).value_counts(normalize=True)*100

Cabin
empty    77.104377
C         6.621773
B         5.274972
D         3.703704
E         3.591470
A         1.683502
F         1.459035
G         0.448934
T         0.112233
Name: proportion, dtype: float64

In [75]:
df_processed['Cabin'] = df_processed['Cabin'].apply(lambda x: 'empty' if str(x)=='empty' else str(x)[0])

rarecategoriesgrouper_cat = RareCategoriesGrouper(columns=['Cabin'])

df_processed = rarecategoriesgrouper_cat.fit_transform(df_processed)

df_processed

,Age,SibSp,Parch,Fare,Embarked,Ticket,Name,Pclass,Sex,Cabin,PassengerId,Survived,Ticket_len
0,22.0,1.0,0.0,7.2500,S,other,mr,3,male,empty,1,0,9
1,38.0,1.0,0.0,65.6344,C,other,mrs,1,female,C,2,1,8
2,26.0,0.0,0.0,7.9250,S,other,miss,3,female,empty,3,1,16
3,35.0,1.0,0.0,53.1000,S,other,mrs,1,female,C,4,1,6
4,35.0,0.0,0.0,8.0500,S,other,mr,3,male,empty,5,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,27.0,0.0,0.0,13.0000,S,other,other,2,male,empty,887,0,6
887,19.0,0.0,0.0,30.0000,S,other,miss,1,female,B,888,1,6
888,28.0,1.0,2.0,23.4500,S,other,miss,3,female,empty,889,0,10
889,26.0,0.0,0.0,30.0000,C,other,mr,1,male,C,890,1,6


#### Анализ Embarked
Тут чисто OHE

In [78]:
df_processed['Embarked'].value_counts(normalize=True)*100

Embarked
S    72.502806
C    18.855219
Q     8.641975
Name: proportion, dtype: float64

#### Итоговый результат

In [81]:
df_processed[list_not_num_ftrs].describe(include=['object'])

,Pclass,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,891,891,891
unique,3,4,2,1,8,3
top,3,mr,male,other,empty,S
freq,491,517,577,891,687,646


In [84]:
for col in list_not_num_ftrs:
    display(pd.get_dummies(df_processed[col], prefix=f'{col}_', drop_first=True))

,Pclass__2,Pclass__3
0,False,True
1,False,False
2,False,True
3,False,False
4,False,True
...,...,...
886,True,False
887,False,False
888,False,True
889,False,False


,Name__mr,Name__mrs,Name__other
0,True,False,False
1,False,True,False
2,False,False,False
3,False,True,False
4,True,False,False
...,...,...,...
886,False,False,True
887,False,False,False
888,False,False,False
889,True,False,False


,Sex__male
0,True
1,False
2,False
3,False
4,True
...,...
886,True
887,False
888,False
889,True


""
0
1
2
3
4
...
886
887
888
889


,Cabin__B,Cabin__C,Cabin__D,Cabin__E,Cabin__F,Cabin__empty,Cabin__other
0,False,False,False,False,False,True,False
1,False,True,False,False,False,False,False
2,False,False,False,False,False,True,False
3,False,True,False,False,False,False,False
4,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...
886,False,False,False,False,False,True,False
887,True,False,False,False,False,False,False
888,False,False,False,False,False,True,False
889,False,True,False,False,False,False,False


,Embarked__Q,Embarked__S
0,False,True
1,False,False
2,False,True
3,False,True
4,False,True
...,...,...
886,False,True
887,False,True
888,False,True
889,False,False


In [85]:
df_processed

,Age,SibSp,Parch,Fare,Embarked,Ticket,Name,Pclass,Sex,Cabin,PassengerId,Survived,Ticket_len
0,22.0,1.0,0.0,7.2500,S,other,mr,3,male,empty,1,0,9
1,38.0,1.0,0.0,65.6344,C,other,mrs,1,female,C,2,1,8
2,26.0,0.0,0.0,7.9250,S,other,miss,3,female,empty,3,1,16
3,35.0,1.0,0.0,53.1000,S,other,mrs,1,female,C,4,1,6
4,35.0,0.0,0.0,8.0500,S,other,mr,3,male,empty,5,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,27.0,0.0,0.0,13.0000,S,other,other,2,male,empty,887,0,6
887,19.0,0.0,0.0,30.0000,S,other,miss,1,female,B,888,1,6
888,28.0,1.0,2.0,23.4500,S,other,miss,3,female,empty,889,0,10
889,26.0,0.0,0.0,30.0000,C,other,mr,1,male,C,890,1,6
